# **Posture Evaluation**

#### The objective of this part of the project is to receive a picture of a person sitted in an office enviroment and give recommendations of chair and table adjustments, so you can set a more personalised & ergonimic workplace.

#### Install **Mediapipe Pose**: This framework can detect and track 33 key landmarks on the human body, including shoulders, elbows, wrists, hips, knees, and ankles.

In [1]:
!pip install mediapipe

#### Install **OpenCV (Open Source Computer Vision Library)** for image and video processing:

In [2]:
!pip install opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np
print("OpenCV version:", cv2.__version__)
print("MediaPipe version:", mp.__version__)

OpenCV version: 4.10.0
MediaPipe version: 0.10.14


In [2]:
# Initialize MediaPipe Pose
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(static_image_mode=True)

I0000 00:00:1723035437.394379 237540271 gl_context.cc:357] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M2


## Calculate the angles

It calculates the angles of hip, knee and ankle to see if they are inside of the recommended range:

In [3]:
# Load the image
image = cv2.imread('images/img_chair_up.png')

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [4]:
# Process the image to find pose landmarks
results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

W0000 00:00:1723035437.458111 237543487 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1723035437.464569 237543487 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
/opt/anaconda3/lib/python3.11/site-packages/google/protobuf/symbol_database.py:55: UserWarning: SymbolDatabase.GetPrototype() is deprecated. Please use message_factory.GetMessageClass() instead. SymbolDatabase.GetPrototype() will be removed soon.
  warnings.warn('SymbolDatabase.GetPrototype() is deprecated. Please '


In [5]:
# Process the image to find pose landmarks
results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

# Function to calculate the angle between three points
def calculate_angle(a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(cosine_angle)
    return np.degrees(angle)

# Extract keypoints (landmarks) and calculate angles if landmarks are detected
if results.pose_landmarks:
    landmarks = results.pose_landmarks.landmark
    
    # Extract left landmarks
    left_shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,
                     landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
    left_elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
    left_wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
    left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,
                landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
    left_knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,
                 landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
    left_ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,
                  landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]

    # Extract right landmarks
    right_shoulder = [landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,
                      landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y]
    right_elbow = [landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value].y]
    right_wrist = [landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value].y]
    right_hip = [landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,
                 landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y]
    right_knee = [landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,
                  landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y]
    right_ankle = [landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,
                   landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y]

    # Calculate angles for the left side
    left_hip_angle = calculate_angle(left_shoulder, left_hip, left_knee)
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)

    # Calculate angles for the right side
    right_hip_angle = calculate_angle(right_shoulder, right_hip, right_knee)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

    # Determine which side is more visible based on confidence scores
    if landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].visibility > landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].visibility:
        print("Detected Left Side")
        hip_angle = left_hip_angle
        knee_angle = left_knee_angle
        elbow_angle = left_elbow_angle
        
        print(f'Hip Angle: {left_hip_angle:.2f} degrees')
        print(f'Knee Angle: {left_knee_angle:.2f} degrees')
        print(f'Elbow Angle: {left_elbow_angle:.2f} degrees')
    else:
        print("Detected Right Side")
        hip_angle = right_hip_angle
        knee_angle = right_knee_angle
        elbow_angle = right_elbow_angle
        print(f'Hip Angle: {right_hip_angle:.2f} degrees')
        print(f'Knee Angle: {right_knee_angle:.2f} degrees')
        print(f'Elbow Angle: {right_elbow_angle:.2f} degrees')

# Display image with pose landmarks (optional)
#mp_drawing = mp.solutions.drawing_utils
#mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
#cv2.imshow('Pose Detection', image)
#cv2.waitKey(0)
#cv2.destroyAllWindows()

Detected Right Side
Hip Angle: 99.06 degrees
Knee Angle: 94.78 degrees
Elbow Angle: 125.21 degrees


## Recommended angles

**Hip**: 90-120 degrees

**Knee**: 90-130 degrees

**Elbows**: 90-120 degrees

In [12]:
import ergonomic_recommendations

NameError: name 'pose' is not defined

In [ ]:
#function for recommendations if the desk is not adjustable:

def ergonomic_analysis_fixed_desk(elbow_angle, hip_angle, knee_angle):
    recommendations = []

    # Case 1: Elbow angle < 90 (Raise chair)
    if elbow_angle < 90:
        if hip_angle > 120 and knee_angle > 130:
            recommendations.append("Raise the chair and use a footrest to correct hip and knee angles.")
        elif hip_angle > 120:
            recommendations.append("Raise the chair and use a footrest to correct hip angle.")
        elif knee_angle > 130:
            recommendations.append("Raise the chair and use a footrest to correct knee angle.")
        else:
            recommendations.append("Raise the chair to increase elbow angle.")

    # Case 2: Elbow angle > 120 (Lower chair)
    elif elbow_angle > 120:
        if hip_angle < 90 and knee_angle < 90:
            recommendations.append("Lower the chair to correct elbow, hip, and knee angles.")
        elif hip_angle < 90:
            recommendations.append("Lower the chair to correct elbow and hip angles.")
        elif knee_angle < 90:
            recommendations.append("Lower the chair to correct elbow and knee angles.")
        else:
            recommendations.append("Lower the chair to decrease elbow angle.")

    # Case 3: Hip angle < 90 (Raise chair)
    if hip_angle < 90 and elbow_angle >= 90 and elbow_angle <= 120:
        if knee_angle < 90:
            recommendations.append("Raise the chair to correct hip and knee angles.")
        else:
            recommendations.append("Raise the chair to increase hip angle.")

    # Case 4: Hip angle > 120 (Lower chair)
    elif hip_angle > 120 and elbow_angle >= 90 and elbow_angle <= 120:
        if knee_angle > 130:
            recommendations.append("Lower the chair to correct hip and knee angles.")
        else:
            recommendations.append("Lower the chair to decrease hip angle.")

    # Case 5: Knee angle < 90 (Raise chair)
    if knee_angle < 90 and elbow_angle >= 90 and elbow_angle <= 120 and hip_angle >= 90 and hip_angle <= 120:
        recommendations.append("Raise the chair to increase knee angle. Use a footrest if needed.")

    # Case 6: Knee angle > 130 (Lower chair)
    elif knee_angle > 130 and elbow_angle >= 90 and elbow_angle <= 120 and hip_angle >= 90 and hip_angle <= 120:
        recommendations.append("Lower the chair to decrease knee angle.")

    # Output recommendations
    if not recommendations:
        return "All angles are within recommended ranges. No adjustments needed."
    else:
        return recommendations

In [ ]:
#function for recommendations if the desk is adjustable:



In [9]:
#determine if the desk is adjustable:

desk_adj = input("Do you have a standing desk (adjustable)? Please answer with 'yes' or 'no'.").lower()

if desk_adj == 'yes':
    #recommendations_desk

elif desk_adj == 'no':
    result = ergonomic_analysis_fixed_desk(elbow_angle, hip_angle, knee_angle)
    for recommendation in result:
        print(recommendation)
    
else:
    print("Please enter a valid answer ('yes' or 'no').")


In [8]:
def ergonomic_analysis(elbow_angle, hip_angle, knee_angle):
    recommendations = []

    # Elbow Angle
    if elbow_angle < 90:
        recommendations.append("Raise chair or lower desk to increase elbow angle.")
    elif elbow_angle > 120:
        recommendations.append("Lower chair or raise desk to decrease elbow angle.")

    # Hip Angle
    if hip_angle < 90:
        recommendations.append("Raise chair to increase hip angle. Consider using a footrest.")
    elif hip_angle > 120:
        recommendations.append("Lower chair to decrease hip angle.")

    # Knee Angle
    if knee_angle < 90:
        recommendations.append("Raise chair to increase knee angle. Ensure feet are flat on the floor or use a footrest.")
    elif knee_angle > 130:
        recommendations.append("Lower chair to decrease knee angle.")

    # Output recommendations
    if not recommendations:
        return "All angles are within recommended ranges. No adjustments needed."
    else:
        return recommendations

result = ergonomic_analysis(elbow_angle, hip_angle, knee_angle)
for recommendation in result:
    print(recommendation)

Lower chair or raise desk to decrease elbow angle.


## Interface

Build an interface to make it acessible for a user to upload a picture and get recommendations

In [ ]:
#streamlit, gradio, flask interfaces